In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/flight-price-prediction/business.csv
/kaggle/input/flight-price-prediction/economy.csv
/kaggle/input/flight-price-prediction/Clean_Dataset.csv


Importing the Dataset.

In [2]:
dataset = pd.read_csv('/kaggle/input/flight-price-prediction/Clean_Dataset.csv')

In [3]:
dataset.head(10)

,Unnamed: 0,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953
1,1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953
2,2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956
3,3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955
4,4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955
5,5,Vistara,UK-945,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.33,1,5955
6,6,Vistara,UK-927,Delhi,Morning,zero,Morning,Mumbai,Economy,2.08,1,6060
7,7,Vistara,UK-951,Delhi,Afternoon,zero,Evening,Mumbai,Economy,2.17,1,6060
8,8,GO_FIRST,G8-334,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.17,1,5954
9,9,GO_FIRST,G8-336,Delhi,Afternoon,zero,Evening,Mumbai,Economy,2.25,1,5954


Cleaning up the dataset

In [4]:
dataset.rename(columns={'class':'ticket_class'},inplace=True)
dataset.drop(dataset.columns[[0,2]],axis=1,inplace=True)

Making sure the data-type for categorical columns in 'Category'

In [5]:
dataset.dtypes

airline              object
source_city          object
departure_time       object
stops                object
arrival_time         object
destination_city     object
ticket_class         object
duration            float64
days_left             int64
price                 int64
dtype: object

In [6]:
dataset.airline = dataset.airline.astype('category')
dataset.source_city = dataset.source_city.astype('category')
dataset.departure_time = dataset.departure_time.astype('category')
dataset.stops = dataset.stops.astype('category')
dataset.arrival_time = dataset.arrival_time.astype('category')
dataset.destination_city = dataset.destination_city.astype('category')
dataset.ticket_class = dataset.ticket_class.astype('category')
dataset.dtypes

airline             category
source_city         category
departure_time      category
stops               category
arrival_time        category
destination_city    category
ticket_class        category
duration             float64
days_left              int64
price                  int64
dtype: object

Converting the categorical variables into dummy binary variables

In [7]:
dataset = pd.get_dummies(dataset,prefix_sep='_',drop_first=True)

In [8]:
dataset.head(10)

,duration,days_left,price,airline_Air_India,airline_GO_FIRST,airline_Indigo,airline_SpiceJet,airline_Vistara,source_city_Chennai,source_city_Delhi,...,arrival_time_Evening,arrival_time_Late_Night,arrival_time_Morning,arrival_time_Night,destination_city_Chennai,destination_city_Delhi,destination_city_Hyderabad,destination_city_Kolkata,destination_city_Mumbai,ticket_class_Economy
0,2.17,1,5953,0,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,1,1
1,2.33,1,5953,0,0,0,1,0,0,1,...,0,0,1,0,0,0,0,0,1,1
2,2.17,1,5956,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,1
3,2.25,1,5955,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,1
4,2.33,1,5955,0,0,0,0,1,0,1,...,0,0,1,0,0,0,0,0,1,1
5,2.33,1,5955,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,1
6,2.08,1,6060,0,0,0,0,1,0,1,...,0,0,1,0,0,0,0,0,1,1
7,2.17,1,6060,0,0,0,0,1,0,1,...,1,0,0,0,0,0,0,0,1,1
8,2.17,1,5954,0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,1,1
9,2.25,1,5954,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,1


Creating the features matrix

In [9]:
Predictors = list(dataset.columns)
Predictors.remove('price')
print(Predictors)

['duration', 'days_left', 'airline_Air_India', 'airline_GO_FIRST', 'airline_Indigo', 'airline_SpiceJet', 'airline_Vistara', 'source_city_Chennai', 'source_city_Delhi', 'source_city_Hyderabad', 'source_city_Kolkata', 'source_city_Mumbai', 'departure_time_Early_Morning', 'departure_time_Evening', 'departure_time_Late_Night', 'departure_time_Morning', 'departure_time_Night', 'stops_two_or_more', 'stops_zero', 'arrival_time_Early_Morning', 'arrival_time_Evening', 'arrival_time_Late_Night', 'arrival_time_Morning', 'arrival_time_Night', 'destination_city_Chennai', 'destination_city_Delhi', 'destination_city_Hyderabad', 'destination_city_Kolkata', 'destination_city_Mumbai', 'ticket_class_Economy']


In [10]:
x = dataset.drop('price',axis=1,inplace=False).values

In [11]:
print(x)

[[ 2.17  1.    0.   ...  0.    1.    1.  ]
 [ 2.33  1.    0.   ...  0.    1.    1.  ]
 [ 2.17  1.    0.   ...  0.    1.    1.  ]
 ...
 [13.83 49.    0.   ...  0.    0.    0.  ]
 [10.   49.    0.   ...  0.    0.    0.  ]
 [10.08 49.    0.   ...  0.    0.    0.  ]]


Creating the dependent variable vector

In [12]:
y = dataset.iloc[:,2].values

In [13]:
print(y)

[ 5953  5953  5956 ... 79099 81585 81585]


Handling missing values

In [14]:
from sklearn.impute import SimpleImputer
#for features vector
imputer = SimpleImputer(missing_values=np.nan,strategy='mean')
imputer.fit(x[:,0:2])
x[:,0:2] = imputer.transform(x[:,0:2])
#for independent variable vector
imputer = SimpleImputer(missing_values=np.nan,strategy='mean')
imputer.fit([y])
y = imputer.transform([y]).flatten()

In [15]:
print(x)

[[ 2.17  1.    0.   ...  0.    1.    1.  ]
 [ 2.33  1.    0.   ...  0.    1.    1.  ]
 [ 2.17  1.    0.   ...  0.    1.    1.  ]
 ...
 [13.83 49.    0.   ...  0.    0.    0.  ]
 [10.   49.    0.   ...  0.    0.    0.  ]
 [10.08 49.    0.   ...  0.    0.    0.  ]]


In [16]:
print(y)

[ 5953.  5953.  5956. ... 79099. 81585. 81585.]


spliting dataset into train and test datasets

In [17]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

In [18]:
print(x_train)

[[ 2.08 22.    0.   ...  0.    0.    1.  ]
 [17.25 40.    0.   ...  0.    0.    0.  ]
 [17.42 41.    0.   ...  0.    0.    0.  ]
 ...
 [ 5.67 23.    0.   ...  0.    0.    1.  ]
 [19.33  2.    1.   ...  0.    0.    0.  ]
 [11.25  6.    0.   ...  0.    1.    1.  ]]


In [19]:
print(x_test)

[[13.67 49.    1.   ...  0.    1.    1.  ]
 [ 6.08 11.    0.   ...  0.    1.    1.  ]
 [ 8.25 23.    0.   ...  1.    0.    1.  ]
 ...
 [ 5.92 31.    0.   ...  0.    0.    0.  ]
 [ 7.42 46.    0.   ...  0.    0.    1.  ]
 [26.58  6.    1.   ...  0.    1.    1.  ]]


In [20]:
print(y_train)

[ 2339. 37985. 68303. ...  3891. 40862.  7958.]


In [21]:
print(y_test)

[ 6048. 13421.  2836. ... 50473.  4099. 10967.]


In [22]:
!pip install dmba

     |████████████████████████████████| 11.8 MB 588 kB/s            


Training the Linear Regression Model

In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from dmba import regressionSummary, exhaustive_search
from dmba import backward_elimination, forward_selection, stepwise_selection
from dmba import adjusted_r2_score, AIC_score, BIC_score

no display found. Using non-interactive Agg backend


In [24]:
fp_lrm = LinearRegression()
fp_lrm.fit(x_train,y_train)

print('Linear Regression Model Details: ')
print()
print('Intercept: ', np.round(fp_lrm.intercept_,2))
print(pd.DataFrame({'Predictors':Predictors,'Coefficients':np.round(fp_lrm.coef_,2)}))

Linear Regression Model Details: 

Intercept:  52537.83
                      Predictors  Coefficients
0                       duration         41.75
1                      days_left       -131.71
2              airline_Air_India        122.28
3               airline_GO_FIRST       1700.70
4                 airline_Indigo       2103.92
5               airline_SpiceJet       2291.61
6                airline_Vistara       4083.93
7            source_city_Chennai        -44.18
8              source_city_Delhi      -1370.00
9          source_city_Hyderabad      -1649.01
10           source_city_Kolkata       1622.87
11            source_city_Mumbai       -185.87
12  departure_time_Early_Morning        876.44
13        departure_time_Evening        759.04
14     departure_time_Late_Night       1837.35
15        departure_time_Morning        861.22
16          departure_time_Night        721.35
17             stops_two_or_more       2071.33
18                    stops_zero      -7593.47
19  

Testing the model

In [25]:
fp_pred = fp_lrm.predict(x_test)

print('Prediction Outcome:')
print()
result = round(pd.DataFrame({
    'Actual': y_test,
    'Predicted': fp_pred,
    'Residual': y_test - fp_pred
}),2)

result.head(10)

Prediction Outcome:



,Actual,Predicted,Residual
0,6048.0,4624.12,1423.88
1,13421.0,9403.79,4017.21
2,2836.0,6072.41,-3236.41
3,1890.0,5647.43,-3757.43
4,56588.0,52543.28,4044.72
5,53152.0,54069.98,-917.98
6,4942.0,3397.24,1544.76
7,54841.0,53530.44,1310.56
8,41269.0,50352.36,-9083.36
9,5120.0,9128.42,-4008.42


Checking model performance

In [26]:
fp_train_pred = fp_lrm.predict(x_train)

#for training dataset
r2 = round(r2_score(y_train,fp_train_pred),3)
adjusted_r2 = round(adjusted_r2_score(y_train,fp_train_pred,fp_lrm),3)
aic = round(AIC_score(y_train,fp_train_pred,fp_lrm),3)
bic = round(BIC_score(y_train,fp_train_pred,fp_lrm),3)

print('Model Performance for training data:')
print('r2: ',r2)
print('adjusted_r2: ',adjusted_r2)
print('aic: ',aic)
print('bic: ',bic)

print()

#for testing dataset
r2 = round(r2_score(y_test,fp_pred),3)
adjusted_r2 = round(adjusted_r2_score(y_test,fp_pred,fp_lrm),3)
aic = round(AIC_score(y_test,fp_pred,fp_lrm),3)
bic = round(BIC_score(y_test,fp_pred,fp_lrm),3)

print('Model Performance for testing data:')
print('r2: ',r2)
print('adjusted_r2: ',adjusted_r2)
print('aic: ',aic)
print('bic: ',bic)

Model Performance for training data:
r2:  0.911
adjusted_r2:  0.911
aic:  4917060.821
bic:  4917393.266

Model Performance for testing data:
r2:  0.912
adjusted_r2:  0.912
aic:  1228294.93
bic:  1228583.014


Checking model accuracy

In [27]:
print('Model accuracy for training data:')
regressionSummary(y_train,fp_train_pred)

print()

print('Model accuracy for testing data:')
regressionSummary(y_test,fp_pred)

Model accuracy for training data:

Regression statistics

                      Mean Error (ME) : -0.0000
       Root Mean Squared Error (RMSE) : 6765.5596
            Mean Absolute Error (MAE) : 4572.2577
          Mean Percentage Error (MPE) : -4.3324
Mean Absolute Percentage Error (MAPE) : 46.4055

Model accuracy for testing data:

Regression statistics

                      Mean Error (ME) : 32.6853
       Root Mean Squared Error (RMSE) : 6707.8498
            Mean Absolute Error (MAE) : 4547.3509
          Mean Percentage Error (MPE) : -3.8712
Mean Absolute Percentage Error (MAPE) : 46.3528


* Since there is not much difference in the model performance and accuracy for Training and Testin datasets this model is not an overfit.

* Also since we are getting very high r2 and adjusted_r2 values for both training and testing dataset this is a very good model.